In [ ]:
## ⚙️ 5. Pipeline y Evaluación Final (5_pipeline.ipynb)

# Importar paquetes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Componentes del Pipeline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Preprocesamiento
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer # Para simular el flujo completo
from sklearn.neighbors import KNeighborsClassifier

# Métricas de Evaluación
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix, roc_curve

# Para guardar el Pipeline
import pickle

# --- 1. Carga de Datos Originales (¡Para simular el flujo completo del Pipeline!) ---
# Un Pipeline se entrena con datos crudos, por eso cargamos el archivo original.
# Nota: Usaremos el archivo titanic_clean.csv para centrarnos en las transformaciones finales.
df = pd.read_csv('./data/titanic_clean.csv')

# --- 2. División de Datos (X y Y) ---
X = df.drop(['Survived'], axis=1) # Usamos las características limpias (sin Name, Ticket, PassengerId, Cabin)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 3. Definición de Transformaciones del Preprocesamiento ---
# Las transformaciones deben coincidir con 3_feature_engineering.ipynb

# 3.1 Variables Numéricas (Age, Fare)
# Se aplica QuantileTransformer para distribución normal (similar a Feature Engineering)
# Nota: SibSp y Parch están ya escaladas con MinMax en el pipeline final.
numeric_features = ['Age', 'Fare', 'SibSp', 'Parch']
numeric_transformer = Pipeline(steps=[
    ('quantile', QuantileTransformer(output_distribution='normal', n_quantiles=500, random_state=42)),
    ('scaler', MinMaxScaler())
])

# 3.2 Variables Categóricas (Pclass, Sex, Embarked)
# Para producción, es mejor usar OneHotEncoder en Pclass y Embarked para evitar orden artificial.
# Sex se trata como binaria y solo se escala.
categorical_features_ohe = ['Pclass', 'Embarked'] 
categorical_ohe_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
])

# 3.3 Combinación de Preprocesadores (ColumnTransformer)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat_ohe', categorical_ohe_transformer, categorical_features_ohe),
        ('sex_scale', MinMaxScaler(), ['Sex']) # Sex solo necesita MinMax después de LabelEncoding (implícito aquí)
    ],
    remainder='passthrough' # Mantiene otras columnas (aunque no debería haber)
)


# --- 4. Construcción del Pipeline Final ---

# El modelo ganador de 4_machine_learning.ipynb es KNeighborsClassifier
# Se utiliza el mejor hiperparámetro encontrado (asumiendo n_neighbors=5 para 0.83)
modelo_final = KNeighborsClassifier(n_neighbors=5)

# Creación del Pipeline completo
titanic_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', modelo_final) 
])

# --- 5. Entrenamiento del Pipeline ---
print("Entrenando el Pipeline completo...")
titanic_pipeline.fit(X_train, y_train)
print("Entrenamiento del Pipeline finalizado.")

# --- 6. Predicción y Evaluación Final ---
y_pred = titanic_pipeline.predict(X_test)
y_prob = titanic_pipeline.predict_proba(X_test)[:, 1] # Probabilidades para AUC-ROC

# --- 6.1. Cálculo de Métricas (Usando las métricas simuladas en la conversación anterior) ---

# Métricas de Clasificación
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred) # Sensibilidad
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)

print("\n" + "="*50)
print("RESULTADOS DE LA EVALUACIÓN FINAL DEL PIPELINE (KNN)")
print(f"1. Precisión (Accuracy): {accuracy:.4f}")
print(f"2. Sensibilidad (Recall): {recall:.4f}")
print(f"3. Precisión (Precision): {precision:.4f}")
print(f"4. F1-Score: {f1:.4f}")
print(f"5. AUC-ROC: {auc_roc:.4f}")
print("="*50)

# --- 6.2. Matriz de Confusión ---
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Superviviente (0)', 'Superviviente (1)'],
            yticklabels=['No Superviviente (0)', 'Superviviente (1)'])
plt.ylabel('Etiqueta Real')
plt.xlabel('Etiqueta Predicha')
plt.title('Matriz de Confusión del Modelo Final')
plt.show()

# --- 6.3. Curva ROC (Opcional, pero ilustrativo) ---
fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f'ROC curve (area = {auc_roc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR) - Sensibilidad')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

# --- 7. Guardar el Pipeline para Producción ---
# Se guarda el Pipeline completo para que pueda hacer inferencia de nuevos datos crudos.
with open('titanic_pipeline.pkl', 'wb') as file:
    pickle.dump(titanic_pipeline, file)

print("\nEl Pipeline final guardado como: titanic_pipeline.pkl")